# Genomeconnect(GC) dataset test: from survey to HPO 

In [3]:
# Use a data to test before officially starting the running
from src.graph.builder import umls_mapping_graph 
result = umls_mapping_graph.invoke({'text': 'Pancreatic cancer','field_type':'short'})
print(result)

✅ enter is_question_mappable_node 
✅ Extracted terms: ['Pancreatic cancer']
🌐 [Pancreatic cancer] API Status: 200
✅ Results for Pancreatic cancer: [{'code': 'HP:0002894', 'term': 'increased risk of pancreatic cancer', 'description': None}]
🧪 Final UMLS mappings: [{'original': 'Pancreatic cancer', 'candidates': [{'code': 'HP:0002894', 'term': 'increased risk of pancreatic cancer', 'description': None}]}]
🧩 Entered rank_mappings_node
🧠 Raw LLM output for 'Pancreatic cancer': [
  {
    "matched_code": "HP:0002894",
    "matched_term": "increased risk of pancreatic cancer",
    "confidence": "100%"
  }
]
✅ Final ranked mappings: {'text': 'Pancreatic cancer', 'field_type': 'short', 'extracted_terms': ['Pancreatic cancer'], 'umls_mappings': [{'original': 'Pancreatic cancer', 'candidates': [{'code': 'HP:0002894', 'term': 'increased risk of pancreatic cancer', 'description': None}]}], 'mappability_retry_count': 0, 'ranked_mappings': [{'original': 'Pancreatic cancer', 'ranked_candidates': [{'co

In [2]:
# import necessary lab
from src.graph.builder import umls_mapping_graph
import requests
import pandas as pd
import logging
from src.config.agents import AGENT_LLM_MAP
from src.prompts.template import apply_prompt_template

1. Reading the test data

In [ ]:
# Load the excel test data
excel_file_path = "gc.xlsx"
df = pd.read_excel(excel_file_path)
print(df.head())

              Section                                           Question  \
0  Mother's Pregnancy  Amniotic fluid issue - Too much fluid (polyhyd...   
1  Mother's Pregnancy                           Decreased fetal movement   
2  Mother's Pregnancy                              Delivery complication   
3  Mother's Pregnancy                            Maternal health problem   
4  Mother's Pregnancy                                    Placental issue   

                            HPO term    HPO code  
0  Abnormality of the amniotic fluid  HP:0001560  
1           Decreased fetal movement  HP:0001558  
2                  Abnormal delivery  HP:0001787  
3      Prenatal maternal abnormality  HP:0002686  
4        Abnormality of the placenta  HP:0100767  


2. Batch process

In [7]:

# Iterate over each row of the DataFrame to map the survey question text to a UMLS code/term
for idx, row in df.iterrows():
    # Source text column and a fixed field type for the agent
    question_text = row['Question']
    field_type = 'radio'  # adjust if your form field varies

    # Defaults to write back even if mapping fails
    agent_term = ""
    agent_code = ""
    confidence = 0.0

    try:
        # Call your LangGraph-based mapping agent
        # Expecting a dict with keys like "refine_mapping" and/or "validated_mappings"
        result = umls_mapping_graph.invoke({
            "text": question_text,
            "field_type": field_type,
            "retries": 0  # keep retries at 0 for deterministic behavior
        })

        # 1) Prefer refined mappings when present (treated as high confidence)
        refine_mapping = result.get("refine_mapping", {})
        if refine_mapping and isinstance(refine_mapping, dict):
            agent_code = refine_mapping.get("refined_code", "")   # <-- fixed typo
            agent_term = refine_mapping.get("refined_term", "")
            confidence = 1.0  # refined result is considered high confidence

        else:
            # 2) Fallback: use the first validated mapping, if available
            validated_mappings = result.get("validated_mappings", [])
            if validated_mappings and isinstance(validated_mappings, list):
                best_mapping = validated_mappings[0]  # assumes the agent returns best-first
                agent_code = best_mapping.get("best_match_code", "")
                agent_term = best_mapping.get("best_match_term", "")
                confidence = best_mapping.get("confidence", 0.0)

                # Normalize confidence to float if the agent returned it as a string
                if isinstance(confidence, str):
                    try:
                        confidence = float(confidence)
                    except ValueError:
                        confidence = 0.0  # fallback if conversion fails

    except Exception as e:
        # Log and continue processing subsequent rows
        print(f"❌ Error on row {idx + 1}: {e}")

    # Write outputs back into the DataFrame
    # (Make sure these columns exist or will be created as new ones.)
    df.at[idx, 'agent_term'] = agent_term
    df.at[idx, 'agent_code'] = agent_code
    df.at[idx, 'confidence'] = confidence

    print(f"✅ Processed row {idx + 1}")



✅ Extracted terms: ['Abnormal amniotic fluid volume']
🌐 [Abnormal amniotic fluid volume] API Status: 404
❌ Failed for term: Abnormal amniotic fluid volume
🧪 Final UMLS mappings: [{'original': 'Abnormal amniotic fluid volume', 'candidates': []}]
✅ Revised terms: ['Abnormality of the amniotic fluid']
🌐 [Abnormality of the amniotic fluid] API Status: 200
✅ Results for Abnormality of the amniotic fluid: [{'code': 'HP:0001560', 'term': 'Abnormality of the amniotic fluid', 'description': 'Abnormality of the amniotic fluid, which is the fluid contained in the amniotic sac surrounding the developing fetus. [https://orcid.org/0000-0002-0736-9199]'}]
🧪 Final UMLS mappings: [{'original': 'Abnormality of the amniotic fluid', 'candidates': [{'code': 'HP:0001560', 'term': 'Abnormality of the amniotic fluid', 'description': 'Abnormality of the amniotic fluid, which is the fluid contained in the amniotic sac surrounding the developing fetus. [https://orcid.org/0000-0002-0736-9199]'}]}]
🧩 Entered rank_

Checking the results

In [ ]:
print(df.head())

,Section,Question,HPO term,HPO code,agent_term,agent_code,confidence
0,Mother's Pregnancy,Amniotic fluid issue - Too much fluid (polyhyd...,Abnormality of the amniotic fluid,HP:0001560,Abnormality of the amniotic fluid,HP:0001560,0.9
1,Mother's Pregnancy,Decreased fetal movement,Decreased fetal movement,HP:0001558,Decreased fetal movement,HP:0001558,1.0
2,Mother's Pregnancy,Delivery complication,Abnormal delivery,HP:0001787,Delivery complication,HP:0001787,1.0
3,Mother's Pregnancy,Maternal health problem,Prenatal maternal abnormality,HP:0002686,,,0.0
4,Mother's Pregnancy,Placental issue,Abnormality of the placenta,HP:0100767,Abnormality of the placenta,HP:0100767,0.9
...,...,...,...,...,...,...,...
182,Oral Health,Malformed teeth,Malformed teeth,HP:0006482,Abnormality of the teeth,HP:0000164,0.9
183,Oral Health,Abnormality of primary teeth (baby teeth probl...,Abnormality of primary teeth,HP:0006481,Abnormality of dental eruption,HP:0006292,0.7
184,Oral Health,Abnormality of the dental root,Abnormality of the dental root,HP:0006486,Abnormality of the dental root,HP:0006486,1.0
185,Oral Health,Abnormality of dental pulp,Abnormality of dental pulp,HP:0006479,Abnormality of the dental pulp,HP:0006479,1.0


3. Export results.

In [12]:
# Output the result and check it 
df.to_excel("mapped_gc.xlsx")